<a href="https://colab.research.google.com/github/kvsdileep/problem_solving/blob/main/Interview_Slot_Assignment_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from datetime import datetime, timedelta

def parse_time(time_str):
    """
    Parses a time string in HH:MM format (24-hour) into a datetime.time object.

    Args:
        time_str (str): The time string to parse.

    Returns:
        datetime.time: The parsed time object.
        Returns None if the string is invalid
    """
    try:
        return datetime.strptime(time_str, '%H:%M').time()
    except ValueError:
        return None  # Handle invalid time format

def generate_time_slots(start_time_str, end_time_str, duration_minutes=40):
    """
    Generates a list of time slots with a fixed duration between a start and end time.

    Args:
        start_time_str (str): The start time in HH:MM format (e.g., "09:00").
        end_time_str (str): The end time in HH:MM format (e.g., "17:00").
        duration_minutes (int, optional): The duration of each time slot in minutes. Defaults to 40.

    Returns:
        list: A list of time slot strings in "HH:MM-HH:MM" format.
              Returns an empty list if start or end time is invalid.
    """
    start_time = parse_time(start_time_str)
    end_time = parse_time(end_time_str)

    if start_time is None or end_time is None:
        return []  # Return empty list for invalid times

    slots = []
    current_time = datetime.combine(datetime.today(), start_time)
    end_time_dt = datetime.combine(datetime.today(), end_time)

    while current_time + timedelta(minutes=duration_minutes) <= end_time_dt:
        slot_start = current_time.strftime('%H:%M')
        slot_end = (current_time + timedelta(minutes=duration_minutes)).strftime('%H:%M')
        slots.append(f"{slot_start}-{slot_end}")
        current_time += timedelta(minutes=duration_minutes)
    return slots

def load_data(slots_file, people_file, calendly_file):
    """
    Loads data from CSV files and returns structured data.  Handles errors
    during file reading and data parsing.

    Args:
        slots_file (str): Path to the slots CSV file.
        people_file (str): Path to the people CSV file.
        calendly_file (str): Path to the calendly CSV file.

    Returns:
        tuple: (time_slots, people, calendly_calls).  Returns (None, None, None)
               if any error occurs during loading or parsing.
    """
    time_slots = {}
    people = {}
    calendly_calls = []

    try:
        with open(slots_file, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                start_time = parse_time(row['StartTime'])
                end_time = parse_time(row['EndTime'])
                if start_time and end_time:  # Only process if times are valid
                  slot_duration = 40
                  slots = generate_time_slots(row['StartTime'], row['EndTime'], slot_duration)
                  for slot in slots:
                    time_slots[slot] = {
                        'capacity': int(row['Capacity']),
                        'interviewers': [],
                        'available': True,
                    }
                else:
                    print(f"Skipping slot with invalid time: {row['StartTime']} - {row['EndTime']}")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error loading or parsing slots file: {e}")
        return None, None, None

    try:
        with open(people_file, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                availability_str = row['Availability']
                # Parse availability string into a list of time ranges
                availability_ranges = [
                    (parse_time(start.strip()), parse_time(end.strip()))
                    for time_range in availability_str.split(',')
                    for start, end in [time_range.split('-')]
                ]
                # Filter out any invalid time ranges (where parsing failed)
                availability_ranges = [(start, end) for start, end in availability_ranges if start and end]

                people[row['Person']] = {
                    'Name': row['Person'],
                    'Availability': availability_ranges,
                    'AIGF': row['AIGF Interviewer'].strip().lower() == 'yes',
                    'Engineering': row['Engineering Interviewer'].strip().lower() == 'yes',
                    'Expertise': row.get('Expertise', ''),  # Added Expertise
                }
    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error loading or parsing people file: {e}")
        return None, None, None

    try:
        with open(calendly_file, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                start_time = parse_time(row['StartTime'])
                if start_time:
                  calendly_calls.append({
                      'Start Time': start_time,
                      'Cx Email': row['Cx Email'],
                      'Field': row['Field'],
                      'Linkedin': row['Linkedin'],
                      'Phone': row['Phone'],
                      'Years of Experience': row['Years of Experience'],
                      'Coding': row['Coding'],
                      'Fellowship Type': 'Engineering' if row['Coding'].strip().lower() == 'yes' else 'Generalist', #added fellowship type
                      'Assigned': False,  # Added for tracking assignment status
                  })
                else:
                    print(f"Skipping calendly call with invalid time: {row['StartTime']}")
        # Sort calendly calls by start time
        calendly_calls.sort(key=lambda x: x['Start Time'])
    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error loading or parsing calendly file: {e}")
        return None, None, None

    return time_slots, people, calendly_calls



def assign_interviews(time_slots, people, calendly_calls):
    """
    Assigns interviewers to calendly calls based on availability, expertise, and fellowship type.

    Args:
        time_slots (dict): Dictionary of time slots.
        people (dict): Dictionary of people (interviewers).
        calendly_calls (list): List of calendly calls.

    Returns:
        tuple: (assigned_calls, unassigned_calls) where each is a list.
    """
    assigned_calls = []
    unassigned_calls = []

    for call in calendly_calls:
        if call['Assigned']:  # Skip if already assigned
            continue
        call_start_time = call['Start Time']
        call_end_time = (datetime.combine(datetime.today(), call_start_time) + timedelta(minutes=40)).time()
        call_slot_str = call_start_time.strftime('%H:%M') + '-' + call_end_time.strftime('%H:%M')


        interview_type = call['Field']
        fellowship_type = call['Fellowship Type']

        for person_name, person in people.items():
            if person['Expertise'] == interview_type:
                # Check for fellowship type availability
                if (fellowship_type == 'Engineering' and person['Engineering']) or \
                   (fellowship_type == 'Generalist' and person['AIGF']):
                    for start, end in person['Availability']:
                         # Convert start and end time to datetime.time for comparison.
                        start_time_dt = datetime.combine(datetime.today(), start)
                        end_time_dt = datetime.combine(datetime.today(), end)
                        call_start_time_dt = datetime.combine(datetime.today(), call_start_time)
                        call_end_time_dt = datetime.combine(datetime.today(), call_end_time)
                        if start_time_dt <= call_start_time_dt and call_end_time_dt <= end_time_dt:
                            if call_slot_str in time_slots and time_slots[call_slot_str]['available']:
                                time_slots[call_slot_str]['interviewers'].append(person_name)
                                time_slots[call_slot_str]['capacity'] -= 1
                                if time_slots[call_slot_str]['capacity'] == 0:
                                    time_slots[call_slot_str]['available'] = False;
                                call['Assigned'] = True
                                assigned_calls.append({
                                    'Time Slot': call_slot_str,
                                    'Interviewer': person_name,
                                    'Callee Email': call['Cx Email'],
                                    'Interview Type': interview_type,
                                    'Fellowship Type': fellowship_type
                                })
                                break #important break, breaks the inner loop
                    if call['Assigned']: # break for outer loop
                        break

        if not call['Assigned']:
            unassigned_calls.append(call)
    return assigned_calls, unassigned_calls

def print_assignments(assigned_calls, unassigned_calls):
    """Prints the assigned and unassigned calls in a readable format.

        Args:
        assigned_calls (list): List of assigned calls.
        unassigned_calls (list): List of unassigned calls.
    """
    print("Assigned Interviews:")
    if not assigned_calls:
        print("No interviews were assigned.")
    else:
        for assignment in assigned_calls:
            print(f"Time Slot: {assignment['Time Slot']}, Interviewer: {assignment['Interviewer']}, "
                  f"Callee Email: {assignment['Callee Email']}, Interview Type: {assignment['Interview Type']}, "
                  f"Fellowship Type: {assignment['Fellowship Type']}")

    print("\nUnassigned Interviews:")
    if not unassigned_calls:
        print("All interviews were assigned successfully.")
    else:
        for call in unassigned_calls:
            print(f"Start Time: {call['Start Time']}, Callee Email: {call['Cx Email']}, Interview Type: {call['Field']}, Fellowship Type: {call['Fellowship Type']}")



def main():
    """
    Main function to run the interview assignment process.
    """
    slots_file = 'Calendly Assignment - Slots.csv'
    people_file = 'Calendly Assignment - People.csv'
    calendly_file = 'Calendly Assignment - Calendly.csv'

    time_slots, people, calendly_calls = load_data(slots_file, people_file, calendly_file)

    if time_slots and people and calendly_calls: # Check if data loaded successfully
        assigned_calls, unassigned_calls = assign_interviews(time_slots, people, calendly_calls)
        print_assignments(assigned_calls, unassigned_calls)
    else:
        print("Failed to load data.  Please check the CSV files and try again.")

if __name__ == "__main__":
    main()